In [4]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_wesad as dr_w
import tools.data_reader_popane as dr_p
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

In [12]:
import statsmodels.api as sm
threshold = "fixed"

In [ ]:
# LOAD TRAIN AND TEST DATA
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dt)

# metrics = train.Metrics.ALL
# metrics = train.Metrics.ECG \
    # + train.Metrics.EDA \
    # + train.Metrics.ANKLE + train.Metrics.WRIST

metrics_list = [
    [ # ECG time
        train.Metrics.BPM, 
        train.Metrics.IBI, 
        train.Metrics.SDNN, 
        train.Metrics.RMSSD, 
    ],
    [ # ECG frequency
        train.Metrics.HF_RR, 
        train.Metrics.LF_RR
    ],
    [ # EDA metrics
        train.Metrics.MEAN_SCL, 
        train.Metrics.SCR_RATE
    ],
    [ # all
        train.Metrics.BPM, 
        train.Metrics.IBI, 
        train.Metrics.SDNN, 
        train.Metrics.RMSSD, 
        train.Metrics.LF_RR,
        train.Metrics.MEAN_SCL, 
        train.Metrics.SCR_RATE
    ]
]

single_metrics = [
    train.Metrics.BPM, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.RMSSD, 
    train.Metrics.LF_RR,
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases = [
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Break"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", 
        "Speech_Relax", "Speech_Anticipate"
    ],
    [
        "Baseline_Rest", 
        "BugBox_Relax",
        "Speech_Relax"
    ],
    ["BugBox_Break", "Speech_Break"],
    ["BugBox_Exposure", "Speech_Exposure"]
]

anxiety_label_type = "Anxiety"

for phases in model_phases:
    print(f"PHASES: {phases} " + "-"*30)
    for metrics in metrics_list:
        print(f"METRICS: {metrics}")
        x, y = train.Train_APD.get_apd_data_ranking(metrics, phases, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold)
        x = x.drop(["phaseId"], axis=1)
        # drop subjects with noisy data
        x = x[x['subject'] != 84.0]
        y = y[y['subject'] != 84.0]
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]

        x.drop(labels=["anxietyGroup"], axis=1)

        # 0-1 scaling
        for i in range(3, len(x.columns)):
            data_col = x[x.columns[i]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x[x.columns[i]] = data_col

        model = LinearRegression()
        model.fit(x, y)
        # print(model.intercept_) 
        # print(model.coef_)
        print(f"R2: {model.score(x, y)}\n")

print("\n")

for phases in model_phases:
    print(f"PHASES: {phases} " + "-"*30)
    for metric in single_metrics:
        print(f"METRICS: {metrics}")
        x, y = train.Train_APD.get_apd_data_ranking(metrics, phases, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold)
        x = x.drop(["phaseId"], axis=1)
        # drop subjects with noisy data
        x = x[x['subject'] != 84.0]
        y = y[y['subject'] != 84.0]
        # x = x[x['subject'] != 8.0]
        # y = y[y['subject'] != 8.0]

        x.drop(labels=["anxietyGroup"], axis=1)

        # 0-1 scaling
        for i in range(3, len(x.columns)):
            data_col = x[x.columns[i]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x[x.columns[i]] = data_col

        model = LinearRegression()
        model.fit(x, y)
        # print(model.intercept_) 
        # print(model.coef_)
        print(f"R2: {model.score(x, y)}\n")